In [1]:
import os
import sys
import re
import json
from pprint import pprint
from dotenv import load_dotenv
import google.generativeai as palm

sys.path.append('..')
from backend.DB.db import Database

In [2]:
load_dotenv()
api_key = os.getenv('PALM_API_KEY')
palm.configure(api_key=api_key)

In [3]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [4]:
db = Database()

sql = '''
SELECT topic_name FROM topics
WHERE category = "hotel";
'''

topics = db.query_sql(sql)
topics = [item[0] for item in topics]

with open(os.path.join('backend', 'gpt-instructions.txt'), 'r') as FILE:
    instructions = FILE.read()

instructions = instructions.replace('%%topic_list%%', str(topics))
print(instructions)

Your task is to analyze and summarize salient points under each topic from
a review. These are your tasks:

* Indicate if a topic is mentioned in the review: true or false

* Evaluate if the guest was angry about that aspect of their stay:  true or false.

* Evaluate the sentiment on a scale of 1 to 5. Scale is:

0 - Not applicable
1 - very negative
2 - negative
3 - neutral
4 - positive
5 - very positive

* Summarize the guest's comments on the topic, in 2 - 3 sentences.

Analyze the review below as instructed above, delimited by triple 
ticks. Format as JSON, exactly as follows. Include all topics and key/value pairs.

{topic : {"Relevant": value, "Angry": value, "Sentiment": value, "Summary": summary}}

Topics = ['amenities', 'frontdesk service', 'hotel condition', 'housekeeping', 'value']



In [6]:
results = db.query('raw_reviews')
review = f"{results[7]['title']}. {results[7]['review_text']}"
pprint(review)
print('==========================\n')
prompt = f'{instructions}\n\n=======================\n\n{review}'

completion = palm.chat(
    messages=[{'author': '0', 'content': prompt}],
    temperature=0
)

pattern = re.compile(r'{(?:[^{}]|((?:{[^{}]*})+))*}')
match = pattern.search(completion.last)
read_review = match.group()

read_review = read_review.replace('\n', '')
read_review = read_review.replace('\\', '')

pprint(json.loads(read_review))

('Roaches everywhere! Avoid!. Our entire building was infested with baby '
 'roaches. They were all over the outside of the building and landing on us as '
 'we entered our room. Once inside, we found that they were all over the '
 'inside of our room in the curtains, bed covers, and air vents.  We were in a '
 'second floor room and they were all the way up to the roofing.  We left as '
 'this was just absolutely disgusting.  I could not imagine closing my eyes '
 'and going to sleep there.  We would not recommend this place even to people '
 'we don’t like!')

{'amenities': {'Angry': True,
               'Relevant': True,
               'Sentiment': 1,
               'Summary': 'The hotel had no amenities. There was no pool, no '
                          'gym, and no restaurant. The only thing they had was '
                          'a vending machine.'},
 'frontdesk service': {'Angry': False,
                       'Relevant': False,
                       'Sentiment': 0,
        